In [1]:
import hail as hl
from gnomad.utils.sparse_mt import split_info_annotation

hl.init(app_name="load_vqsr")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/submit/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-02-04 14:56:50 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-02-04 14:56:52 WARN  Client:69 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Running on Apache Spark version 3.1.2
SparkUI available at http://dice01-mlx.mlx:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/Hail-joint-caller/notebooks/hail-20220204-1456-0.2.68-13190f0b6103.log


In [2]:
default_reference = "GRCh38"
path_to_mt = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/HailVQSR/output/VQSR.filtered.final.vcf.gz"
output_ht_path = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/load_VQSR_test_results.ht"

In [3]:
mt = hl.import_vcf(
    path_to_mt, 
    reference_genome=default_reference,
    force_bgz=True
)

In [4]:
ht = mt.rows()

In [5]:
ht.info.AS_SB_TABLE.show() # The str field needs to be separated into integers

2022-02-04 14:57:27 Hail: INFO: Coerced sorted dataset==>           (4 + 1) / 5]


,,
locus,alleles,
locus<GRCh38>,array<str>,str
chr1:10146,"[""AC"",""A""]","""3,1|6,12"""
chr1:10291,"[""C"",""T""]","""0,3|0,2"""
chr1:10390,"[""CCCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA"",""C""]","""9,8|3,3"""
chr1:10397,"[""C"",""CCCCTAA""]","""0,0|4,1"""
chr1:10439,"[""AC"",""A""]","""8,8|3,7"""
chr1:10442,"[""CCT"",""C""]","""10,14|1,2"""
chr1:10443,"[""CT"",""C""]","""4,7|2,1"""
chr1:10450,"[""T"",""C""]","""3,5|2,1"""


In [6]:
ht.info.AS_VQSLOD.show() # Similarly, the array of strings needs to be remapped to an array of floats

2022-02-04 14:57:41 Hail: INFO: Coerced sorted dataset              (2 + 3) / 5]


,,
locus,alleles,
locus<GRCh38>,array<str>,array<str>
chr1:10146,"[""AC"",""A""]","[""-2.0694""]"
chr1:10291,"[""C"",""T""]","[""-38.7260""]"
chr1:10390,"[""CCCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA"",""C""]","[""-2.8748""]"
chr1:10397,"[""C"",""CCCCTAA""]","[""0.1832""]"
chr1:10439,"[""AC"",""A""]","[""-2.3830""]"
chr1:10442,"[""CCT"",""C""]","[""-2.7520""]"
chr1:10443,"[""CT"",""C""]","[""5.6990""]"
chr1:10450,"[""T"",""C""]","[""-17.7967""]"


In [7]:
ht = ht.annotate(
    info=ht.info.annotate(
        AS_VQSLOD=ht.info.AS_VQSLOD.map(hl.float),
        AS_SB_TABLE=ht.info.AS_SB_TABLE.split(r'\|').map(
            lambda x: hl.if_else(
                x == '', hl.missing(hl.tarray(hl.tint32)), x.split(',').map(hl.int)
            )
        ),
    ),
)

In [8]:
ht = hl.split_multi_hts(ht)

In [9]:
ht = ht.annotate(
        info=ht.info.annotate(**split_info_annotation(ht.info, ht.a_index)),
)


In [10]:
ht = ht.annotate(
    filters=ht.filters.union(hl.set([ht.info.AS_FilterStatus])),
)

In [11]:
ht.write(output_ht_path, overwrite=True)

2022-02-04 14:57:52 Hail: INFO: Coerced sorted dataset==>           (4 + 1) / 5]
2022-02-04 14:58:03 Hail: INFO: Coerced sorted dataset==>           (4 + 1) / 5]
2022-02-04 14:59:07 Hail: INFO: wrote table with 10782643 rows in 5 partitions to file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/load_VQSR_test_results.ht
    Total size: 840.74 MiB
    * Rows: 840.74 MiB
    * Globals: 11.00 B
    * Smallest partition: 2135068 rows (167.69 MiB)
    * Largest partition:  2167633 rows (168.39 MiB)
